In [1]:
import requests, json
import pandas as pd
import datetime as dt
from twilio.rest import Client #version 6.0.0

In [2]:
df_credentials = pd.read_csv("credentials.txt", sep ="," ) #Keep your credentials in a text file

In [3]:
#read the credentials required to access apis
weatherAPIKey = df_credentials['value'][0]
account_sid = df_credentials['value'][1]
auth_token = df_credentials['value'][2]
from_phone = df_credentials['value'][3]
to_phone = df_credentials['value'][4]

<h1> Access open weather API to get the weather Forecast </h1>

In [4]:
#open weather api - You need to signup with then to get the API KEY to access their API
url = 'http://api.openweathermap.org/data/2.5/forecast'

In [5]:
def getCityID(city):
    params = {'q':city, 'APPID':df_credentials['value'][0] }
    response = requests.get(url, params = params)
    data = json.loads(response.content)
    return data['city']['id']

In [6]:
#to get the ID of a city
#getCityID('Alpharetta,US')

In [7]:
#to get the forecast for 5 days in a dataframe
def getWeatherForecast(cityid, appid,units):
    url = 'http://api.openweathermap.org/data/2.5/forecast'
    params = {'id':cityid,'APPID':appid, 'units':units}
    response = requests.get(url, params = params)
    data = json.loads(response.content)
    weather_list = data['list']
    
    df_list = []
    for elem in weather_list:
        #print(elem)
        dict_ = {}
        dict_['timestamp'] = elem['dt_txt']
        dict_['max_temp'] = elem['main']['temp_max']
        dict_['min_temp'] = elem['main']['temp_min']
    
        description = []
        desc_list = elem['weather']
        for l in desc_list:
            description.append(l['description']) 
        dict_['description'] = ','.join(description)
        df_list.append(dict_)
        
    df = pd.DataFrame(df_list)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['max_temp'] = df['max_temp'].astype(int)
    df['min_temp'] = df['min_temp'].astype(int)
    
    return df

<h1> Send SMS using Twilio with today's weather forecast </h1>

In [8]:
#send SMS using Twilio Trial account
def sendSMS(account_sid, auth_token, from_phone, to_phone, msg):
    client = Client(account_sid, auth_token)
    client.messages.create(
        to=to_phone,
        from_=from_phone,
        body=msg)

<h3> Get weather forecast for my location and send as an sms </h3>

In [9]:
df = getWeatherForecast(4179574,weatherAPIKey,'imperial' )

In [10]:
#Today's Weather Forecast
df_today = df[df['timestamp'].dt.date == dt.date.today()]
df_today['timestamp'] = df_today['timestamp'].dt.strftime("%I:%M %p")

C:\Users\KRISHNACHAITANYA\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:
#Get the sms message to send
message = df_today[['description','timestamp','max_temp']][0:2].to_string(index = False)

In [12]:
#send SMS
sendSMS(account_sid, auth_token, from_phone, to_phone,message )